# predict

In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics, model_selection
import lightgbm as lgb
import os

In [3]:
# if test_mode is True, just run training and cross-validation on training data;
# if False, also make predictions on test set
test_mode = False

In [4]:
shared_data_path = r'C:\\Users\\Chris\\Documents\\code_kaggle_plasticc___shared_data\\PLAsTiCC-2018'

In [5]:
# read data
all_meta = pd.read_hdf(os.path.join(shared_data_path, 'features', 'all_data.hdf5'), key='file0')
train_meta_approx = pd.read_hdf(os.path.join(shared_data_path, 'features', 'train_meta_approx.hdf5'), key='file0')
train_meta_exact = pd.read_hdf(os.path.join(shared_data_path, 'features', 'train_meta_exact.hdf5'), key='file0')


In [6]:
# map classes to range [0, 14]
classes = np.sort(all_meta.loc[all_meta['target'].notnull(), 'target'].unique().astype(int))

In [12]:
# Train separate models for galatic and extra-galactic, since these classes contain disjoint sets of objects,
# and can be distinguished by whether hostgl_photoz == 0, as observed here:
# https://www.kaggle.com/kyleboone/naive-benchmark-galactic-vs-extragalactic?scriptVersionId=6104036#
galactic_bool = all_meta['hostgal_photoz'] == 0
exact_bool = all_meta['hostgal_photoz'].notnull().astype(int)
galactic_classes = np.sort(all_meta.loc[all_meta['target'].notnull() & galactic_bool, 'target'].unique().astype(int))
non_galactic_classes = np.sort(all_meta.loc[all_meta['target'].notnull() & ~galactic_bool, 'target'].unique().astype(int))

In [13]:
galactic_classes

array([ 6, 16, 53, 65, 92])

In [14]:
non_galactic_classes

array([15, 42, 52, 62, 64, 67, 88, 90, 95])

In [15]:
# transform the target so the classes are the integers range(num_classes)
for df in [all_meta, train_meta_approx, train_meta_exact]:
    df['target_trans'] = np.nan
    df['target_trans_galactic'] = np.nan
    df['target_trans_non_galactic'] = np.nan
    for k, class_list in enumerate([classes, galactic_classes, non_galactic_classes]):
        if k == 0:
            suffix = ''
        elif k == 1:
            suffix = '_galactic'
        else:
            suffix = '_non_galactic'
        for i in range(len(class_list)):
            df.loc[df['target'] == class_list[i], 'target_trans' + suffix] = i


In [16]:

# train 2 models for each class, one for when we have exact redshift, and another for when we don't
train_cols_exact_redshift \
    = [c for c in train_meta_exact.columns if
      c not in ['object_id', 'ra', 'decl', 'gal_l', 'gal_b', 'target', 'target_trans', 'target_trans_galactic',
                 'target_trans_non_galactic', 'ddf',
                 'distmod', 'mwebv', 'hostgal_photoz', 'hostgal_photoz_err', 'index']]
train_cols_approx_redshift \
    = [c for c in train_meta_approx.columns if
      c not in ['object_id', 'ra', 'decl', 'gal_l', 'gal_b', 'target', 'target_trans', 'target_trans_galactic',
                 'target_trans_non_galactic', 'ddf',
                 'distmod', 'mwebv', 'hostgal_specz', 'index']]


In [17]:

# separate parameters for galactic and non-galactic
params_galactic = {'boosting_type': 'gbdt', 'application': 'binary', 'num_leaves': 32, 'seed': 0, 'verbose': -1,
                   'min_data_in_leaf': 1, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'lambda_l1': 0, 'lambda_l2': 1,
                   'learning_rate': 0.02}
params_non_galactic = {'boosting_type': 'gbdt', 'application': 'binary', 'num_leaves': 16, 'seed': 0, 'verbose': -1,
                       'min_data_in_leaf': 1, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'lambda_l1': 0,
                       'lambda_l2': 1, 'learning_rate': 0.02}

In [20]:
#num_rounds = 3000
num_rounds = 3

In [21]:
# cross-validate on train set, and measure distribution of out-of-sample predicted values
train_err_exact = []
test_err_exact = []
train_err_approx = []
test_err_approx = []
cv = model_selection.KFold(5, shuffle=True, random_state=4)
galactic_bool_train = train_meta_exact['hostgal_photoz'] == 0
train_meta_exact['predict_max_exact'] = 0
train_meta_exact['predict_max_approx'] = 0
train_meta_approx['predict_max_exact'] = 0
train_meta_approx['predict_max_approx'] = 0
predict_cols = ['class_' + str(c) for c in classes]
train_prediction_exact \
    = pd.DataFrame(np.zeros((len(train_meta_exact), 14)), index=train_meta_exact.index, columns=predict_cols)
train_prediction_approx \
    = pd.DataFrame(np.zeros((len(train_meta_exact), 14)), index=train_meta_exact.index, columns=predict_cols)
eval_prediction_exact \
    = pd.DataFrame(np.zeros((len(train_meta_exact), 14)), index=train_meta_exact.index, columns=predict_cols)
eval_prediction_approx \
    = pd.DataFrame(np.zeros((len(train_meta_exact), 14)), index=train_meta_exact.index, columns=predict_cols)
importance = {}
best_iter_exact = {c: [] for c in classes}
best_iter_approx = {c: [] for c in classes}

In [22]:
# Evaluate accuracy on resampled training set having similar distribution to test. The data note says
# "The training data are mostly composed of nearby, low-redshift, brighter objects while the test data contain
# more distant (higher redshift) and fainter objects."  So we resample to achieve a similar distribution of
# hostgal_photoz.
train_bool = all_meta['target'].notnull()
ddf = all_meta['ddf'] == 1
w = pd.DataFrame(index=train_meta_exact.index, columns=['galactic', 'non_galactic'])
w['galactic'] = galactic_bool_train.astype(int)
w['non_galactic'] = np.nan
bands = np.arange(all_meta.loc[~train_bool, 'hostgal_photoz'].min(),
                  all_meta.loc[~train_bool, 'hostgal_photoz'].max() + 0.00001, 0.1)
for i in range(len(bands[:-1])):
    band_bool = ~galactic_bool_train & ~ddf & (train_meta_exact['hostgal_photoz'] >= bands[i]) \
                & (train_meta_exact['hostgal_photoz'] <= bands[i + 1])
    train_prop = band_bool.sum() / (~galactic_bool_train & ~ddf).sum()
    test_prop = ((all_meta.loc[~train_bool & ~galactic_bool, 'hostgal_photoz'] >= bands[i])
                 & (all_meta.loc[~train_bool & ~galactic_bool, 'hostgal_photoz'] <= bands[i + 1])).sum() \
                / (~train_bool & ~galactic_bool).sum()
    w.loc[band_bool, 'non_galactic'] = test_prop / train_prop
w.loc[ddf] = 0

In [23]:
for train_ind, test_ind in list(cv.split(train_meta_exact.index, train_meta_exact['target_trans'])):
    train_bool = train_meta_exact.index.isin(train_ind)
    ddf = train_meta_exact['ddf'] == 1

    for i, c in enumerate(classes):
        g = c in galactic_classes
        gal_bool_train_curr = galactic_bool_train == g
        params = params_galactic if g else params_non_galactic
        col = 'class_' + str(c)
        weight_col = 'galactic' if g else 'non_galactic'

        # exact redshift model
        lgb_train = lgb.Dataset(train_meta_exact.loc[train_bool & gal_bool_train_curr, train_cols_exact_redshift],
                                label=(train_meta_exact.loc[train_bool & gal_bool_train_curr, 'target'] == c).astype(int))
        lgb_valid = lgb.Dataset(train_meta_exact.loc[(~train_bool) & gal_bool_train_curr & ~ddf, train_cols_exact_redshift],
                                label=(train_meta_exact.loc[(~train_bool) & gal_bool_train_curr & ~ddf, 'target'] == c).astype(int),
                                weight=w.loc[(~train_bool) & gal_bool_train_curr & ~ddf, weight_col])
        est = lgb.train(train_set=lgb_train, valid_sets=[lgb_train, lgb_valid], valid_names=['train', 'valid'],
                        params=params, num_boost_round=num_rounds, early_stopping_rounds=100)
        best_iter_exact[c].append(est.best_iteration)
        train_prediction_exact.loc[~train_bool & gal_bool_train_curr, col] = est.predict(
            train_meta_exact.loc[(~train_bool) & gal_bool_train_curr, train_cols_exact_redshift],
            num_iteration=est.best_iteration)
        # measure errors on train and test
        eval_prediction_exact.loc[gal_bool_train_curr, col] \
            = est.predict(train_meta_exact.loc[gal_bool_train_curr, train_cols_exact_redshift],
                          num_iteration=est.best_iteration)

        # approx redshift models
        lgb_train = lgb.Dataset(train_meta_approx.loc[train_bool & gal_bool_train_curr, train_cols_approx_redshift],
                                label=(train_meta_approx.loc[train_bool & gal_bool_train_curr, 'target'] == c).astype(int))
        lgb_valid = lgb.Dataset(train_meta_approx.loc[(~train_bool) & gal_bool_train_curr & ~ddf, train_cols_approx_redshift],
                                label=(train_meta_approx.loc[(~train_bool) & gal_bool_train_curr & ~ddf, 'target'] == c).astype(int),
                                weight=w.loc[(~train_bool) & gal_bool_train_curr & ~ddf, weight_col])
        est = lgb.train(train_set=lgb_train, valid_sets=[lgb_train, lgb_valid], valid_names=['train', 'valid'],
                        params=params, num_boost_round=num_rounds, early_stopping_rounds=100)
        best_iter_approx[c].append(est.best_iteration)
        train_prediction_approx.loc[(~train_bool) & gal_bool_train_curr, col] = est.predict(
            train_meta_approx.loc[(~train_bool) & gal_bool_train_curr, train_cols_approx_redshift],
            num_iteration=est.best_iteration)
        # measure errors on train and test
        eval_prediction_approx.loc[gal_bool_train_curr, col] \
            = est.predict(train_meta_approx.loc[gal_bool_train_curr, train_cols_approx_redshift],
                          num_iteration=est.best_iteration)

        imp_arr = est.feature_importance()
        importance[g] = {c: imp_arr[i] for i, c in enumerate(train_cols_approx_redshift)}

    # fill nulls
    for df in [train_prediction_exact, train_prediction_approx, eval_prediction_exact, eval_prediction_approx]:
        df[predict_cols] = df[predict_cols].fillna(0)

    # scale so columns add to 1
    for df in [eval_prediction_exact, eval_prediction_approx]:
        col_sum = df.sum(axis=1)
        for c in df.columns:
            df[c] /= col_sum

    train_err_exact.append(metrics.log_loss(train_meta_exact.loc[train_bool & ~ddf, 'target_trans'],
                                      eval_prediction_exact.loc[train_bool & ~ddf, predict_cols]))
    test_err_exact.append(metrics.log_loss(train_meta_exact.loc[~train_bool & ~ddf, 'target_trans'],
                                     eval_prediction_exact.loc[~train_bool & ~ddf, predict_cols]))
    print('Train exact error: ', train_err_exact)
    print('Test exact error: ', test_err_exact)
    train_err_approx.append(metrics.log_loss(train_meta_approx.loc[train_bool & ~ddf, 'target_trans'],
                                      eval_prediction_approx.loc[train_bool & ~ddf, predict_cols]))
    test_err_approx.append(metrics.log_loss(train_meta_approx.loc[~train_bool & ~ddf, 'target_trans'],
                                     eval_prediction_approx.loc[~train_bool & ~ddf, predict_cols]))
    print('Train approx error: ', train_err_approx)
    print('Test approx error: ', test_err_approx)

with open('log.txt', 'w') as f:
    f.write('Train exact error: '.format(train_err_exact))
    f.write('\nTest exact error: '.format(test_err_exact))
    f.write('\nTrain approx error: {}'.format(train_err_approx))
    f.write('\nTest approx error: {}'.format(test_err_approx))
    f.write('\nMean exact train error: {}'.format(np.mean(train_err_exact)))
    f.write('\nMean exact test error: {}'.format(np.mean(test_err_exact)))
    f.write('\nMean approx train error: {}'.format(np.mean(train_err_approx)))
    f.write('\nMean approx test error: {}'.format(np.mean(test_err_approx)))


[1]	train's binary_logloss: 0.225388	valid's binary_logloss: 0.29855
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.215764	valid's binary_logloss: 0.290397
[3]	train's binary_logloss: 0.207074	valid's binary_logloss: 0.28322
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.207074	valid's binary_logloss: 0.28322
[1]	train's binary_logloss: 0.225388	valid's binary_logloss: 0.29855
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.215764	valid's binary_logloss: 0.290397
[3]	train's binary_logloss: 0.207074	valid's binary_logloss: 0.28322
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.207074	valid's binary_logloss: 0.28322
[1]	train's binary_logloss: 0.291493	valid's binary_logloss: 0.353168
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.284421	valid's binary_logloss: 0.343257
[3]	train's binary_loglo

[1]	train's binary_logloss: 0.218922	valid's binary_logloss: 0.428341
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.206341	valid's binary_logloss: 0.402135
[3]	train's binary_logloss: 0.19608	valid's binary_logloss: 0.381087
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.19608	valid's binary_logloss: 0.381087
[1]	train's binary_logloss: 0.672303	valid's binary_logloss: 0.6764
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.664857	valid's binary_logloss: 0.669018
[3]	train's binary_logloss: 0.657927	valid's binary_logloss: 0.661548
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.657927	valid's binary_logloss: 0.661548
[1]	train's binary_logloss: 0.673379	valid's binary_logloss: 0.678636
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.666748	valid's binary_logloss: 0.672623
[3]	train's binary_log

[1]	train's binary_logloss: 0.0869013	valid's binary_logloss: 0.0414282
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.0821412	valid's binary_logloss: 0.0403759
[3]	train's binary_logloss: 0.0782743	valid's binary_logloss: 0.0397071
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.0782743	valid's binary_logloss: 0.0397071
[1]	train's binary_logloss: 0.662659	valid's binary_logloss: 0.656252
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.64564	valid's binary_logloss: 0.641059
[3]	train's binary_logloss: 0.629245	valid's binary_logloss: 0.626471
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.629245	valid's binary_logloss: 0.626471
[1]	train's binary_logloss: 0.662659	valid's binary_logloss: 0.656252
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.64564	valid's binary_logloss: 0.641059
[3]	train's 

[1]	train's binary_logloss: 0.147448	valid's binary_logloss: 0.102562
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.146318	valid's binary_logloss: 0.101982
[3]	train's binary_logloss: 0.145241	valid's binary_logloss: 0.101816
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.145241	valid's binary_logloss: 0.101816
[1]	train's binary_logloss: 0.0682172	valid's binary_logloss: 0.0575471
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.0646913	valid's binary_logloss: 0.0543117
[3]	train's binary_logloss: 0.0616647	valid's binary_logloss: 0.0515399
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.0616647	valid's binary_logloss: 0.0515399
[1]	train's binary_logloss: 0.0682172	valid's binary_logloss: 0.0575471
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.0646913	valid's binary_logloss: 0.0543117
[3]	tr

[1]	train's binary_logloss: 0.293684	valid's binary_logloss: 0.32564
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.286498	valid's binary_logloss: 0.316521
[3]	train's binary_logloss: 0.279845	valid's binary_logloss: 0.308845
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.279845	valid's binary_logloss: 0.308845
[1]	train's binary_logloss: 0.293684	valid's binary_logloss: 0.32564
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.286498	valid's binary_logloss: 0.316521
[3]	train's binary_logloss: 0.279834	valid's binary_logloss: 0.30891
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.279834	valid's binary_logloss: 0.30891
[1]	train's binary_logloss: 0.656458	valid's binary_logloss: 0.666791
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.63951	valid's binary_logloss: 0.651003
[3]	train's binary_logl

[1]	train's binary_logloss: 0.67313	valid's binary_logloss: 0.692007
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.666776	valid's binary_logloss: 0.684971
[3]	train's binary_logloss: 0.660362	valid's binary_logloss: 0.678482
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.660362	valid's binary_logloss: 0.678482
[1]	train's binary_logloss: 0.318205	valid's binary_logloss: 0.298311
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.304393	valid's binary_logloss: 0.28674
[3]	train's binary_logloss: 0.292592	valid's binary_logloss: 0.277458
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.292592	valid's binary_logloss: 0.277458
[1]	train's binary_logloss: 0.318205	valid's binary_logloss: 0.298311
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.304393	valid's binary_logloss: 0.28674
[3]	train's binary_lo

[1]	train's binary_logloss: 0.0880056	valid's binary_logloss: 0.0373753
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.0842898	valid's binary_logloss: 0.0366353
[3]	train's binary_logloss: 0.0803579	valid's binary_logloss: 0.0358815
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.0803579	valid's binary_logloss: 0.0358815
[1]	train's binary_logloss: 0.664123	valid's binary_logloss: 0.648139
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.647303	valid's binary_logloss: 0.633035
[3]	train's binary_logloss: 0.630842	valid's binary_logloss: 0.618094
Did not meet early stopping. Best iteration is:
[3]	train's binary_logloss: 0.630842	valid's binary_logloss: 0.618094
[1]	train's binary_logloss: 0.664123	valid's binary_logloss: 0.648139
Training until validation scores don't improve for 100 rounds
[2]	train's binary_logloss: 0.647303	valid's binary_logloss: 0.633035
[3]	train'

In [25]:
# if test_mode is True, just run training and cross-validation on training data;
# if False, also make predictions on test set
test_mode = False


In [26]:

if not test_mode:
    prediction = pd.DataFrame(np.zeros((len(all_meta), 14)), columns=predict_cols, index=all_meta.index)
    exact_bool = all_meta['hostgal_specz'].notnull()
    train_bool = all_meta['target'].notnull()

    for i, c in enumerate(classes):
        g = c in galactic_classes
        gal_bool_train_curr = galactic_bool_train == g
        galactic_bool_curr = galactic_bool == g
        params = params_galactic if g else params_non_galactic
        col = 'class_' + str(c)

        if not g:
            # model for exact redshift, only needed for non-galactic objects, since all galactic objects in the
            # test set have approx data
            lgb_train = lgb.Dataset(train_meta_exact.loc[gal_bool_train_curr, train_cols_exact_redshift],
                                    label=(train_meta_exact.loc[gal_bool_train_curr, 'target'] == c).astype(int))
            est_exact_redshift = lgb.train(train_set=lgb_train, valid_sets=[lgb_train], valid_names=['train'],
                                          params=params, num_boost_round=int(np.max(best_iter_exact[c])))
            prediction.loc[exact_bool & galactic_bool_curr, col] \
                = est_exact_redshift.predict(all_meta.loc[exact_bool & galactic_bool_curr, train_cols_exact_redshift])

        # model for approx redshift
        lgb_train = lgb.Dataset(train_meta_approx.loc[gal_bool_train_curr, train_cols_approx_redshift],
                                label=(train_meta_approx.loc[gal_bool_train_curr, 'target'] == c).astype(int))
        est_approx_redshift = lgb.train(train_set=lgb_train, valid_sets=[lgb_train], valid_names=['train'],
                                        params=params, num_boost_round=int(np.max(best_iter_approx[c])))
        prediction.loc[~exact_bool & galactic_bool_curr, col] \
            = est_approx_redshift.predict(all_meta.loc[~exact_bool & galactic_bool_curr, train_cols_approx_redshift])

    # fill nulls
    prediction[predict_cols] = prediction[predict_cols].fillna(0)

    # We will calculate the probability that the object is class 99 using 1 - max(other columns). But this is an
    # overestimate, since the max is always less than 1, even in the training set. So adjust for this by comparing to
    # the max in the training set, resampled as before to account for the different distribution. Different
    # distributions for each combination of galactic/non-galactic and approx/exact. All galactic objects have approx
    # redshift, so only need three combinations. First add the target column.
    train_prediction_exact['target'] = train_meta_exact['target']
    train_prediction_approx['target'] = train_meta_approx['target']
    train_approx_galactic_resample \
        = train_prediction_approx.sample(n=10000, weights=w['galactic'], replace=True, random_state=0)
    train_exact_non_galactic_resample \
        = train_prediction_exact.sample(n=10000, weights=w['non_galactic'], replace=True, random_state=0)
    train_approx_non_galactic_resample \
        = train_prediction_approx.sample(n=10000, weights=w['non_galactic'], replace=True, random_state=0)

    # predict class 99
    prediction['class_99'] = 1 - prediction[predict_cols].max(axis=1)
    # adjust as described above
    for exact in [True, False]:
        for g in [True, False]:
            if exact and g:
                continue
            prediction_ind = ~train_bool & (exact_bool == exact) & (galactic_bool == g)
            if exact:
                train_avg_max = train_exact_non_galactic_resample[predict_cols].max(axis=1).mean()
            else:
                if g:
                    train_avg_max = train_approx_galactic_resample[predict_cols].max(axis=1).mean()
                else:
                    train_avg_max = train_approx_non_galactic_resample[predict_cols].max(axis=1).mean()
            if (1 - train_avg_max) < prediction.loc[prediction_ind, 'class_99'].mean():
                old_avg = prediction.loc[prediction_ind, 'class_99'].mean()
                new_avg = old_avg - (1 - train_avg_max)
                prediction.loc[prediction_ind, 'class_99'] *= new_avg / old_avg

    # write output
    prediction[predict_cols + ['class_99']] = prediction[predict_cols + ['class_99']]
    prediction['object_id'] = all_meta['object_id']
    filename = 'Prediction_raw.csv'
    prediction.loc[~train_bool, ['object_id'] + predict_cols + ['class_99']].to_csv(filename, index=False, header=True)


[1]	train's binary_logloss: 0.229686
[2]	train's binary_logloss: 0.220087
[3]	train's binary_logloss: 0.211705
[1]	train's binary_logloss: 0.293803
[2]	train's binary_logloss: 0.286782
[3]	train's binary_logloss: 0.280593
[1]	train's binary_logloss: 0.293803
[2]	train's binary_logloss: 0.286782
[3]	train's binary_logloss: 0.280689
[1]	train's binary_logloss: 0.654245
[2]	train's binary_logloss: 0.637333
[3]	train's binary_logloss: 0.621292
[1]	train's binary_logloss: 0.517984
[2]	train's binary_logloss: 0.514407
[3]	train's binary_logloss: 0.510844
[1]	train's binary_logloss: 0.518444
[2]	train's binary_logloss: 0.515628
[3]	train's binary_logloss: 0.51271
[1]	train's binary_logloss: 0.144203
[2]	train's binary_logloss: 0.143262
[3]	train's binary_logloss: 0.142122
[1]	train's binary_logloss: 0.144417
[2]	train's binary_logloss: 0.143332
[3]	train's binary_logloss: 0.142389
[1]	train's binary_logloss: 0.0652092
[2]	train's binary_logloss: 0.0616696
[3]	train's binary_logloss: 0.0591359